In [313]:
import pandas as pd
import json
import requests
import folium

In [314]:
incidentsdf = pd.read_csv('incidents_may.csv', usecols=["id", "priority", "implicated", "incident_time", "latitude", "longitude"])

#drop duplicates
incidentsdf = incidentsdf.drop_duplicates(subset="id", keep="first")

#cast incident_time to datetime
incidentsdf["incident_time"] = pd.to_datetime(incidentsdf["incident_time"])

#fill nan to 0 and cast implicated to int
incidentsdf["implicated"] = incidentsdf["implicated"].fillna(0)
incidentsdf["implicated"] = incidentsdf["implicated"].astype(int)

#calculate intervals and create column
new = incidentsdf.groupby(pd.Grouper(key="incident_time", freq='60Min')).apply(lambda x: x['incident_time'])
incidentsdf["interval"] = new.index.get_level_values(0)

#multiindex by interval, then id
incidentsdf = incidentsdf.set_index(["interval", "id"])

#print head
incidentsdf.head()

priority  latitude  longitude  implicated  \
interval            id                                                 
2022-04-30 23:00:00 57518         2  4.705279 -74.127686           2   
                    57519         1  4.600571 -74.143379           3   
2022-05-01 00:00:00 57520         1  4.647777 -74.137123           1   
                    57521         1  4.720405 -74.075302           2   
                    57522         1  4.614258 -74.167252           2   

                                incident_time  
interval            id                         
2022-04-30 23:00:00 57518 2022-04-30 23:59:00  
                    57519 2022-04-30 23:54:00  
2022-05-01 00:00:00 57520 2022-05-01 00:08:00  
                    57521 2022-05-01 00:10:00  
                    57522 2022-05-01 00:11:00

In [315]:
agentsdf = pd.read_csv('agents_may.csv', usecols=["dev_id", "time_stamp", "latitude", "longitude"])

#cast time_stamp to datetime
agentsdf["time_stamp"] = pd.to_datetime(agentsdf["time_stamp"])

#calculate intervals and create column
new = agentsdf.groupby(pd.Grouper(key="time_stamp", freq='60Min')).apply(lambda x: x['time_stamp'])
agentsdf["interval"] = new.index.get_level_values(0)

#multiindex by interval, then id
agentsdf = agentsdf.set_index(["interval", "dev_id"])

#print head
agentsdf.head()

time_stamp  latitude  longitude
interval   dev_id                                                  
2022-05-01 868033050089715 2022-05-01 00:01:40  4.551646 -74.108515
           868033050102534 2022-05-01 00:01:42  4.650030 -74.147322
           868033050090085 2022-05-01 00:01:47  4.581688 -74.128348
           868033050103508 2022-05-01 00:01:52  4.551098 -74.147635
           868033050101650 2022-05-01 00:01:55  4.619840 -74.159668

In [316]:
agentsdf.groupby(pd.Grouper(key="time_stamp", freq='5Min'))

In [317]:
incidents_locations = incidentsdf[["longitude","latitude"]].apply(lambda x: ','.join(x.values.astype(str)), axis=1)
incidents_locations.head()

interval             id   
2022-04-30 23:00:00  57518       -74.127685546875,4.705278873443604
                     57519    -74.14337921142578,4.6005706787109375
2022-05-01 00:00:00  57520    -74.13712310791016,4.6477766036987305
                     57521     -74.07530212402344,4.720405101776123
                     57522     -74.16725158691406,4.614258289337158
dtype: object

In [318]:
agents_locations = agentsdf[["longitude","latitude"]].apply(lambda x: ','.join(x.values.astype(str)), axis=1)
agents_locations.head()

interval    dev_id         
2022-05-01  868033050089715     -74.108515,4.5516463
            868033050102534    -74.1473218,4.6500299
            868033050090085    -74.1283484,4.5816876
            868033050103508    -74.1476347,4.5510981
            868033050101650      -74.1596683,4.61984
dtype: object

In [319]:
incidents_sample = incidents_locations.sample(1000)
agents_sample = agents_locations.sample(1000)

In [320]:
folium_map = folium.Map(
    location=[4.6534649, -74.0836453], zoom_start=12, tiles="CartoDB positron"
)

incidents_feature_group = folium.FeatureGroup(name="Incidents")
for i in range(len(incidents_sample)):
    marker = folium.CircleMarker(
        location=incidents_sample.iloc[i].split(",")[::-1],
        radius=1,
        color="red",
        fill=True,
        fill_opacity=0.3,
    )
    marker.add_to(incidents_feature_group)

agents_feature_group = folium.FeatureGroup(name="Agents")
for i in range(len(agents_sample)):
    marker = folium.CircleMarker(
        location=agents_sample.iloc[i].split(",")[::-1],
        radius=1,
        color="blue",
        fill=True,
        fill_opacity=0.3,
    )
    marker.add_to(agents_feature_group)

incidents_feature_group.add_to(folium_map)
agents_feature_group.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map


In [321]:
agents_coordinates = ";".join(agents_sample)
incidents_coordinates = ";".join(incidents_sample)
coordinates = agents_coordinates + ";" + incidents_coordinates

agents_count = agents_sample.shape[0]
sources = ";".join(str(i) for i in range(agents_count))

incidents_count = incidents_sample.shape[0]
destinations = ";".join(
    str(i) for i in range(agents_count, agents_count + incidents_count)
)

#Ojo con correr esto sin estar cirriendo el servicio.
durations = json.loads(
    requests.get(
        f"http://127.0.0.1:5000/table/v1/driving/{coordinates}?sources={sources}&destinations={destinations}"
    ).text
)["durations"]


In [322]:
#Finally, we can create a dataframe with the durations in seconds.}

df = pd.DataFrame(durations)
#df = df.apply(lambda x: x/60)

df = df.set_index(agents_sample.index.get_level_values(1))
df = df.set_axis(list(incidents_sample.index.get_level_values(1)), axis=1)

df


,62704,67183,61322,62661,63179,62454,61439,61213,66695,64748,...,64424,65764,62919,64677,67018,59453,63082,64763,57761,67547
dev_id,,,,,,,,,,,,,,,,,,,,,
868033050090168,644.4,488.7,125.8,679.9,289.8,674.1,114.1,765.7,604.9,1341.4,...,970.8,700.4,1062.4,803.8,1069.6,569.8,349.1,297.9,719.6,597.7
868033050103623,989.4,1026.9,1275.4,946.4,1331.1,1077.2,1263.9,1740.9,1707.7,2252.3,...,1881.7,1050.6,1973.3,1726.5,853.2,1089.0,1470.7,1476.4,1015.2,1485.4
868033050100645,698.3,481.5,497.4,655.3,661.4,735.5,428.8,891.7,730.9,1656.5,...,1285.9,759.5,1377.5,1175.4,767.9,797.9,493.9,499.6,724.1,531.1
868033050140096,1134.5,978.8,592.1,1170.0,756.1,1164.2,594.0,396.3,340.3,1183.1,...,812.5,1190.5,904.1,1112.8,1532.8,1059.9,501.1,449.9,1209.7,936.6
868033050089848,728.1,572.4,209.5,763.6,373.5,757.8,186.2,849.4,688.6,1425.1,...,1054.5,784.1,1146.1,887.5,1067.8,653.5,432.8,381.6,803.3,589.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868033050139395,1248.6,1255.7,1246.2,1205.6,1410.2,1295.5,1177.6,1651.9,1491.1,2416.7,...,2046.1,1309.8,2137.7,1885.1,1196.3,1348.2,1033.1,1030.8,1274.4,815.2
868033050089509,295.5,553.4,580.5,302.8,636.3,375.9,569.0,1108.3,1121.6,1619.7,...,1249.1,250.7,1340.7,995.8,815.2,353.6,879.7,828.5,370.7,1034.3
868033050100496,481.9,343.4,648.6,438.9,704.4,275.6,637.1,1176.4,1189.7,1687.8,...,1317.2,543.1,1408.8,1030.2,767.9,545.9,947.8,896.6,507.7,1102.4
